In [1]:
import multiprocess as mp
import os
from functools import partial
from itertools import product
from typing import Iterable, List, Union

import tqdm
from graph_state.graph_state import *

In [2]:
def print_statistics_for_parallelized_experiments(results):
    print("\n--- Experiment Run Summary ---")
    saved_count = sum(1 for r in results if r.startswith("Saved"))
    skipped_count = sum(1 for r in results if r.startswith("Skipped"))
    failed_count = sum(1 for r in results if r.startswith("Failed"))
    print(f"Successfully saved: {saved_count}")
    print(f"Skipped (existed):  {skipped_count}")
    print(f"Failed:             {failed_count}")
    if failed_count > 0:
        print("\nFailures:")
        for r in results:
            if r.startswith("Failed"):
                print(f"  - {r}")

In [3]:
def _run_bell_sampling_worker(combination: tuple, num_repeats: int, output_dir: str, overwrite: bool) -> str:
    """
    A single-process worker function for bell_sampling_fidelity_experiment.
    'combination' is a tuple: (g: GraphState, err: error model (str), fidelity: float, shots: int, stab_factor: string indicating how many stabilizer elements we want to pick)
    The process will run and save the data to the output_dir.

    Returns:
        A status string for logging.
    """
    g, err, fid, shots, stab_factor = combination

    if stab_factor == '2n':
        numstab = g.n * 2
    elif stab_factor == 'n^2':
        numstab = g.n ** 2
    elif stab_factor.isdigit():
        numstab = int(stab_factor)
    else:
        return f"Failed (param error): Stabilizer factor '{stab_factor}' not recognized"

    # Construct filename and check for overwrite
    filename = f"bell_fidelity_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}_numstab_{numstab}.npy"
    filepath = os.path.join(output_dir, filename)
    
    if not overwrite and os.path.exists(filepath):
        return f"Skipped (exists): {filename}"
    
    # Run the actual experiment
    try:
        # all_fidelities = [fidelity_estimation_via_random_sampling_bitpacked(g, numstab, samples)]
        all_fidelities = []
        for seed_i in range(num_repeats):
            samples = bell_sampling(g, err, fid, shots, seed=seed_i)
            est_fidelity = fidelity_estimation_via_random_sampling_bitpacked(g, numstab, samples)
            all_fidelities.append(est_fidelity)
            
        # Save the result
        np.save(filepath, np.array(all_fidelities))
        return f"Saved ({len(all_fidelities)} repeats): {filename}"
    except Exception as e:
        return f"Failed (runtime error): {filename} with error: {e}"

def bell_sampling_fidelity_experiment(
    graphs: Union[GraphState, List[GraphState]],
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    stabilizer_factors: Union[str, List[str]],
    output_dir: str,
    overwrite: bool = False
):
    """
    Runs a Bell sampling fidelity experiment in parallel for all combinations of parameters. 
    Each combination is run in a separate process.
    Arguments can be passed in as either a single instance or as a list and will be expanded.
    """
    # Normalize all inputs to be lists
    graphs_list = [graphs] if isinstance(graphs, GraphState) else list(graphs)
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)
    stabilizer_factors_list = [stabilizer_factors] if isinstance(stabilizer_factors, str) else list(stabilizer_factors)

    # Create the directory for saving results
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(graphs_list, err_models, fidelities, shots_list, stabilizer_factors_list))
    
    print(f"Starting Bell random sampling experiment for {len(graphs_list)} graph(s), "
          f"{len(err_models)} error model(s), {len(fidelities)} fidelity value(s), "
          f"{len(shots_list)} shot setting(s), and {len(stabilizer_factors_list)} stabilizer factor(s).")
    print(f"Total combinations (jobs) to run: {len(combinations)}")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_bell_sampling_worker,
        num_repeats=num_repeats,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [4]:
def _run_tomo_worker(combination: tuple, g: GraphState, overlap_observables: bool, num_repeats: int, output_dir: str, overwrite: bool) -> str:
    """
    A single-process worker function for partial_tomo_experiment.
    'combination' is a tuple: (err, fid, total_shots)
    
    Returns:
        A status string for logging.
    """
    try:
        err, fid, total_shots = combination
        N = 2 ** g.n

        filename = f"tomo_{g.n}q_F{fid:.3f}_err_{err}_shots_{total_shots}.npy"
        filepath = os.path.join(output_dir, filename)

        if not overwrite and os.path.exists(filepath):
            return f"Skipped (exists): {filename}"

        all_diags_for_combo = []
        for seed_i in range(num_repeats):
            exps = dge_combined(g, err, fid, total_shots, overlap_observables=overlap_observables, seed=seed_i)
            exps = np.maximum(0, exps)
            diags = get_diagonals_from_all_stabilizer_observables(g, exps)
            all_diags_for_combo.append(diags)
            
        # Stack the results into a single 2D NumPy array
        stacked_diags = np.array(all_diags_for_combo) # shape (num_repeats, number_of_diagonals)

        # 5. Save the result
        np.save(filepath, stacked_diags)
        return f"Saved ({stacked_diags.shape}): {filename}"
    
    except Exception as e:
        # Provide more context in the error message
        return f"Failed: {filename} with error: {e}"

def partial_tomo_experiment_parallelized(
    g: GraphState,
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    total_shots: Union[int, Iterable[int]],
    overlap_observables: bool,
    num_repeats: int,
    output_dir: str,
    overwrite: bool,
):
    """
    Runs a partial tomography experiment in parallel for all combinations
    of parameters. Each combination is run in a separate process.

    Args:
        g (GraphState): The graph state object.
        err_model (Union[str, List[str]]): A single error model string or a list of them.
        fidelity (Union[float, Iterable[float]]): A single fidelity value or an iterable.
        total_shots (Union[int, Iterable[int]]): A single total_shot (count) value, 
                                                 or an iterable of values.
        num_repeats (int): The number of times to repeat the experiment
                           for each parameter combination.
        output_dir (str): The directory where the output .npy files will be saved.
        overwrite (bool): If False, skips the calculation if the output file
                          already exists. If True, it will always run and overwrite
                          any existing file.
    """
    # Normalize all inputs to be lists
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [total_shots] if isinstance(total_shots, int) else list(total_shots)

    # Create the directory for saving results
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(err_models, fidelities, shots_list))
    
    print(f"Starting tomography experiment for {g.n} qubits ({len(err_models)} error model(s), "
          f"{len(fidelities)} fidelity value(s), and {len(shots_list)} shot setting(s).")
    print(f"Total combinations (jobs) to run: {len(combinations)}")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_tomo_worker,
        g=g,
        overlap_observables=overlap_observables,
        num_repeats=num_repeats,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [5]:
def _run_bell_sampling_diagonal_worker(combination: tuple, g: GraphState, num_repeats: int, output_dir: str, overwrite: bool) -> str:
    # Unpack the combination tuple
    err, fid, shots = combination

    # Construct filename and check for overwrite
    filename = f"bell_diag_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}.npy"
    filepath = os.path.join(output_dir, filename)

    if not overwrite and os.path.exists(filepath):
        return f"Skipped (exists): {filename}"

    actual_shots = (shots + 1) // 2
    bell_samples = bell_sampling(g, err, fid, actual_shots * num_repeats, seed=num_repeats)
    samples_split = np.split(bell_samples, num_repeats)

    all_diags_for_combo = []

    for samples in samples_split:
        exps = np.array(
            [
                expectation_value_of_observables_from_bell_bitpacked(
                    np.packbits(stab, bitorder="little"), samples
                )
                for stab in g.generate_all_int_staiblizers()
            ]
        )
        sqrt_exps_safe = np.sqrt(np.maximum(0, exps))
        diags = get_diagonals_from_all_stabilizer_observables(g, sqrt_exps_safe)
        all_diags_for_combo.append(diags)

    stacked_diags = np.array(all_diags_for_combo) # Shape (num_repeats, number_of_diagonals)
    np.save(filepath, stacked_diags)
    return f"Saved ({stacked_diags.shape}): {filename}"

def bell_sampling_diagonal_experiment(
    g: GraphState,
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    output_dir: str,
    overwrite: bool,
):
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)

    # Create the directory for saving results if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(err_models, fidelities, shots_list))
    
    print(f"Starting Bell diagonal experiment for {g.n} qubits, {len(err_models)} error model(s), {len(fidelities)} fidelity value(s), and {len(shots_list)} shot setting(s).")
    print(f"Starting experiment for {len(combinations)} parameter combinations...")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_bell_sampling_diagonal_worker,
        num_repeats=num_repeats,
        g=g,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [6]:
def _run_bell_sampling_diagonal_single_run_worker(combination: tuple[GraphState, str, float, int, int], output_dir: str, overwrite: bool) -> str:
    g, err, fid, shots, repeat_idx = combination

    # Construct filename and check for overwrite
    filename = f"bell_scalability_{g.n}q_F{fid:.3f}_err_{err}_shots_{shots}_repeat_{repeat_idx}.npy"
    filepath = os.path.join(output_dir, filename)

    if not overwrite and os.path.exists(filepath):
        return f"Skipped (exists): {filename}"

    actual_shots = (shots + 1) // 2
    bell_samples = bell_sampling(g, err, fid, actual_shots, seed=1000 * g.n + repeat_idx)

    exps = np.array(
        [
            expectation_value_of_observables_from_bell_bitpacked(
                np.packbits(stab, bitorder="little"), bell_samples
            )
            for stab in g.generate_all_int_staiblizers()
        ]
    )
    sqrt_exps_safe = np.sqrt(np.maximum(0, exps))
    diags = get_diagonals_from_all_stabilizer_observables(g, sqrt_exps_safe)

    np.save(filepath, diags)
    return f"Saved ({diags.shape}): {filename}"

def bell_diagonal_scalability_experiment(
    graphs: Union[GraphState, List[GraphState]],
    err_model: Union[str, List[str]],
    fidelity: Union[float, Iterable[float]],
    num_shots: Union[int, Iterable[int]],
    num_repeats: int,
    output_dir: str,
    overwrite: bool,
):
    # Normalize all inputs to be lists
    graphs_list = [graphs] if isinstance(graphs, GraphState) else list(graphs)
    err_models = [err_model] if isinstance(err_model, str) else list(err_model)
    fidelities = [fidelity] if isinstance(fidelity, (float, int)) else list(fidelity)
    shots_list = [num_shots] if isinstance(num_shots, int) else list(num_shots)
    repeat_indices = [i for i in range(num_repeats)]

    # Create the directory for saving results
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving experiment data to: '{output_dir}/'")

    # Create all combinations of the parameters
    combinations = list(product(graphs_list, err_models, fidelities, shots_list, repeat_indices))
    
    print(f"Starting Bell random sampling experiment for {len(graphs_list)} graph(s), "
          f"{len(err_models)} error model(s), {len(fidelities)} fidelity value(s), "
          f"{len(shots_list)} shot setting(s), and {num_repeats} trials.")
    print(f"Total combinations (jobs) to run: {len(combinations)}")
    
    # Set up the partial function for the worker
    worker_partial = partial(
        _run_bell_sampling_diagonal_single_run_worker,
        output_dir=output_dir,
        overwrite=overwrite
    )
    
    # Set up and run the multiprocessing pool
    num_processes = max(1, mp.cpu_count() - 2) # Leave some cores free
    print(f"Running on {num_processes} processes...")

    results = []
    with mp.Pool(processes=num_processes) as pool:
        for result in tqdm.tqdm(pool.imap_unordered(worker_partial, combinations), total=len(combinations), desc="Running Experiments"):
            results.append(result)

    print_statistics_for_parallelized_experiments(results)

In [7]:
"""
We generate the data for 8 qubit graph here.
    1. vary from F=0.5 to F=1 with N = 10^4
    2. vary from 10^2 to 10^5 with F = 0.9 (plotting later)
    X-axis will be log scale so we need to do log scaling X-axis (N)
"""

g = GraphState(8, "complete")
shots = 10_000
# err_model = "single-qubit-dephasing"
err_model = "depolarizing"
repeat = 1000
OVERWRITE = False

# first experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    fidelity=0.9,
    total_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    overlap_observables=False,
    num_repeats=repeat,
    output_dir="data-new/depolarizing/dge_group_nonoverlapped",
    overwrite=False,
)

# second experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    overlap_observables=False,
    num_repeats=repeat,
    output_dir="data-new/depolarizing/dge_group_nonoverlapped",
    overwrite=False,
)

# first experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    fidelity=0.9,
    total_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    overlap_observables=True,
    num_repeats=repeat,
    output_dir="data-new/depolarizing/dge_group_overlapped",
    overwrite=False,
)

# second experiment
partial_tomo_experiment_parallelized(
    g,
    err_model,
    np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    shots,
    overlap_observables=True,
    num_repeats=repeat,
    output_dir="data-new/depolarizing/dge_group_overlapped",
    overwrite=False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    fidelity=0.9,
    num_shots=np.round(np.logspace(start=2, stop=5, base=10, endpoint=True, num=50)).astype(int),
    num_repeats=repeat,
    output_dir="data-new/depolarizing/bsqn",
    overwrite=False,
)

bell_sampling_diagonal_experiment(
    g,
    err_model,
    fidelity=np.round(np.arange(0.5, 1.01, step=0.02), decimals=3),
    num_shots=shots,
    num_repeats=repeat,
    output_dir="data-new/depolarizing/bsqn",
    overwrite=False,
)

# scalability experiment
for n in range(2, 16):
    g = GraphState(n, "complete")
    partial_tomo_experiment_parallelized(
        g,
        err_model,
        0.9,
        2 * 10_000,
        overlap_observables=True,
        num_repeats=25,
        output_dir="data-new/scalability/dge_group_overlapped",
        overwrite=False,
    )
bell_diagonal_scalability_experiment(
    [GraphState(n, "complete") for n in range(2, 24)],
    err_model,
    fidelity=0.9,
    num_shots=20_000,
    num_repeats=25,
    output_dir="data-new/scalability/bsqn",
    overwrite=False,
)

Saving experiment data to: 'data-new/depolarizing/dge_group_nonoverlapped/'
Starting tomography experiment for 8 qubits (1 error model(s), 1 fidelity value(s), and 50 shot setting(s).
Total combinations (jobs) to run: 50
Running on 12 processes...


Running Experiments: 100%|██████████| 50/50 [01:09<00:00,  1.39s/it]


--- Experiment Run Summary ---
Successfully saved: 45
Skipped (existed):  0
Failed:             5

Failures:
  - Failed: tomo_8q_F0.900_err_depolarizing_shots_133.npy with error: Not enough shots to split between all stabilizers (shots given: 133; total circuits to run 192)
  - Failed: tomo_8q_F0.900_err_depolarizing_shots_115.npy with error: Not enough shots to split between all stabilizers (shots given: 115; total circuits to run 192)
  - Failed: tomo_8q_F0.900_err_depolarizing_shots_153.npy with error: Not enough shots to split between all stabilizers (shots given: 153; total circuits to run 192)
  - Failed: tomo_8q_F0.900_err_depolarizing_shots_100.npy with error: Not enough shots to split between all stabilizers (shots given: 100; total circuits to run 192)
  - Failed: tomo_8q_F0.900_err_depolarizing_shots_176.npy with error: Not enough shots to split between all stabilizers (shots given: 176; total circuits to run 192)
Saving experiment data to: 'data-new/depolarizing/dge_group_


Running Experiments: 100%|██████████| 26/26 [00:42<00:00,  1.62s/it]


--- Experiment Run Summary ---
Successfully saved: 26
Skipped (existed):  0
Failed:             0
Saving experiment data to: 'data-new/depolarizing/dge_group_overlapped/'
Starting tomography experiment for 8 qubits (1 error model(s), 1 fidelity value(s), and 50 shot setting(s).
Total combinations (jobs) to run: 50
Running on 12 processes...


...


--- Experiment Run Summary ---
Successfully saved: 0
Skipped (existed):  1
Failed:             0
Saving experiment data to: 'data-new/scalability/dge_group_overlapped/'
Starting tomography experiment for 15 qubits (1 error model(s), 1 fidelity value(s), and 1 shot setting(s).
Total combinations (jobs) to run: 1
Running on 12 processes...



Running Experiments: 100%|██████████| 1/1 [00:00<00:00, 1776.49it/s]


--- Experiment Run Summary ---
Successfully saved: 0
Skipped (existed):  1
Failed:             0
Saving experiment data to: 'data-new/scalability/bsqn/'
Starting Bell random sampling experiment for 22 graph(s), 1 error model(s), 1 fidelity value(s), 1 shot setting(s), and 25 trials.
Total combinations (jobs) to run: 550
Running on 12 processes...



Running Experiments: 100%|██████████| 550/550 [1:44:52<00:00, 11.44s/it]   


--- Experiment Run Summary ---
Successfully saved: 550
Skipped (existed):  0
Failed:             0
